# Parse annotation responses Notebook

Once annotations are done by our annotators, we want to parse the JSON formatted responses from the specialized software Label Studio into a combined dataset.

In [1]:
# import os
# import sys; sys.path.append("./libraries/")
# import pandas as pd
# import numpy as np
# from fleiss_kappa import fleiss_kappa
# from label_studio_JSON_parser import parse_file
import imports as i
import importlib
importlib.reload(i)

c:\Users\Veronii\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


<module 'imports' from 'c:\\Users\\Veronii\\Desktop\\data-wild-west\\code\\imports.py'>

In [2]:
# JSON formatted responses path
path = "../Annotations/annotators_results/"

# Container for individual annotation responses datasets
dfs = []

# Look at the JSON files, parse and join
for file in i.os.listdir(path):
    if file.endswith(".json"):
        # Use our custom function to parse the response file
        df = i.parse_file(path + file)
        # Append to the container
        dfs.append(df)

# Join all files
annotations = i.pd.concat(dfs).reset_index(drop=True)

print(f"A total of {annotations.shape[0]} are now joined.")

A total of 1008 are now joined.


In [3]:
annotations

,ID,text,annotator,Not Determined,Staff,Equipment,Hygiene,Location
0,3,"Sweet nice young people working, they have a s...",344f1158-Bogdan.csv,1.0,1.0,NaN,NaN,NaN
1,4,Has trained for many years here and Bornholmsv...,344f1158-Bogdan.csv,1.0,1.0,1.0,NaN,NaN
2,6,I exercise early in the morning and start befo...,344f1158-Bogdan.csv,0.0,NaN,NaN,NaN,NaN
3,7,there is a nice atmosphere in the morning ther...,344f1158-Bogdan.csv,1.0,NaN,NaN,NaN,NaN
4,8,"I trained 4-6 times a week for several years, ...",344f1158-Bogdan.csv,1.0,1.0,1.0,NaN,NaN
...,...,...,...,...,...,...,...,...
1003,604,Old and No ergonomic equipment for free weight...,0fe50f89-Veron.csv,-1.0,NaN,-1.0,NaN,NaN
1004,607,Its nice Gym to work out its clean and well or...,0fe50f89-Veron.csv,1.0,NaN,NaN,1.0,NaN
1005,608,Its a nice and clean gym💪. Bra personal. I lov...,0fe50f89-Veron.csv,1.0,NaN,NaN,1.0,NaN
1006,609,One the my favourite place!!!,0fe50f89-Veron.csv,1.0,NaN,NaN,NaN,NaN


### Calculate Inter Annotator Agreement

In [4]:
# The categories are in the columns (except the first two: "ID" and "text")
categories = annotations.columns[3:]
# The possible labels are 1.0 (Positive), 0.0 (Neutral), -1.0 (Negative) or NAN (if no sentiment)
labels = [1.0, 0.0, -1.0, i.np.nan]

IAA = i.fleiss_kappa(annotations, categories, labels=labels)

print(f"The Fleiss Kappa for IAA is {IAA:.2f}.")

# for each category we count how many times each of the 4 labels (1.0, 0.0, -1.0, NaN) was assigned
# every 4 columns is assigned to a category in the order they are in annoation table
# columns order is like this : (1.0, 0.0, -1.0, NaN) and gets repeated * 5 (5 categories)

(100, 24)
The Fleiss Kappa for IAA is 0.25.


### Calculate Inter Annotator Agreement for each category

In [5]:
# The categories are in the columns (except the first two: "ID" and "text")
categories = annotations.columns[3:]
# The possible labels are 1.0 (Positive), 0.0 (Neutral), -1.0 (Negative) or np.nan (if no sentiment)
labels = [1.0, 0.0, -1.0, i.np.nan]

IAA = i.fleiss_kappa_by_category(annotations, categories, labels=labels)

# printing Fleiss Kappa for each category
for category, kappa in IAA.items():
    print(f"The Fleiss Kappa for the category '{category}' is {kappa:.2f}.")
    
# in this case in calculating fleiss kappa for each category  
#   1) we have 100 reviews, 
#   2) with each column corresponding to one of the labels (1.0, 0.0, -1.0, NaN)
#   3) the value in each cell indicates the number of raters who assigned that label to the review for that category

The Fleiss Kappa for the category 'annotator' is nan.
The Fleiss Kappa for the category 'Not Determined' is 0.76.
The Fleiss Kappa for the category 'Staff' is 0.83.
The Fleiss Kappa for the category 'Equipment' is 0.82.
The Fleiss Kappa for the category 'Hygiene' is 1.00.
The Fleiss Kappa for the category 'Location' is 0.66.
